# Libraries

In [50]:
import json
import nltk
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD
from nltk.stem import WordNetLemmatizer
import numpy as np
import random
import pickle
from textblob import TextBlob
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from textblob import TextBlob
import os
from autocorrect import Speller

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


# Load JSON Files

In [3]:
# English Data
data_en = json.loads(open('data_en.json').read())

# Spanish Data
data_es = json.loads(open('data_es.json').read())

## Natural Language Processing

In [4]:
intents_en = data_en['interactionModel']['languageModel']

intents_es = data_es['interactionModel']['languageModel']


In [5]:
# Create a list of tuples that assigns a list of words to an intent in English
words_en = []
labels_en = []
documents_en = []
punc_en = ['?', '!', ',', '.']

for intent in intents_en['intent']: 
    for sample in intent['samples']:
        word_list_en = nltk.word_tokenize(sample)
        words_en.extend(word_list_en)
        documents_en.append((word_list_en, intent['name']))
        if intent['name'] not in labels_en:
            labels_en.append(intent['name'])


In [6]:
# Create a list of tuples that assigns a list of words to an intent in Spanish
words_es = []
labels_es = []
documents_es = []
punc_es = ['?', '!', ',', '.']

for intent in intents_es['intent']: 
    for sample in intent['samples']:
        word_list_es = nltk.word_tokenize(sample)
        words_es.extend(word_list_es)
        documents_es.append((word_list_es, intent['name']))
        if intent['name'] not in labels_es:
            labels_es.append(intent['name'])


## Bag of Words

In [7]:
# import NLP library
from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()

In [8]:
# turn words to lower case and to base form
words_en = [lem.lemmatize(word.lower()) for word in words_en if word not in punc_en]
words_es = [lem.lemmatize(word.lower()) for word in words_es if word not in punc_es]


# sort words and remove duplicates in English
words_en = sorted(set(words_en))
labels_en = sorted(set(labels_en))

# sort words and remove duplicates in Spanish
words_es = sorted(set(words_es))
labels_es = sorted(set(labels_es))


## Preprocessing input and output data 

In [9]:
# create training and output arrays in English
training_en = []
output_empty_en = [0] * len(labels_en)
for document in documents_en:
    bag_en = []
    word_patterns_en = document[0]
    word_patterns_en = [lem.lemmatize(word.lower()) for word in word_patterns_en]
    for word in words_en:
        if word in word_patterns_en:
            bag_en.append(1)
        else:
            bag_en.append(0)
            
    # copy output list
    output_en = list(output_empty_en)
    output_en[labels_en.index(document[1])] = 1
    training_en.append([bag_en, output_en])


In [10]:
# create training and output arrays in Spanish
training_es = []
output_empty_es = [0] * len(labels_es)
for document in documents_es:
    bag_es = []
    word_patterns_es = document[0]
    word_patterns_es = [lem.lemmatize(word.lower()) for word in word_patterns_es]
    for word in words_es:
        if word in word_patterns_es:
            bag_es.append(1)
        else:
            bag_es.append(0)
            
    # copy output list
    output_es = list(output_empty_es)
    output_es[labels_es.index(document[1])] = 1
    training_es.append([bag_es, output_es])


In [11]:
# randomizing the order of the training data in English
random.shuffle(training_en)
training_en = np.array(training_en)

train_x_en = list(training_en[:,0])
train_y_en = list(training_en[:,1])

# randomizing the order of the training data in Spanish
random.shuffle(training_es)
training_es = np.array(training_es)

train_x_es = list(training_es[:,0])
train_y_es = list(training_es[:,1])


C:\Users\aashi\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\aashi\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # Remove the CWD from sys.path while we load stuff.


## Build Model

In [12]:
def model_en():
    model = Sequential()
    model.add(Dense(128, input_shape=(len(train_x_en[0]),), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(train_y_en[0]), activation='softmax'))
    return model


def model_es():
    model = Sequential()
    model.add(Dense(128, input_shape=(len(train_x_es[0]),), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(train_y_es[0]), activation='softmax'))
    return model


### Train English Model 

In [13]:
model_en = model_en()

optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_en.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
checkpoint_en = ModelCheckpoint(filepath='model_en.h5', monitor='acc')
model_en.fit(np.array(train_x_en), np.array(train_y_en), epochs=500, batch_size=5, verbose=1, callbacks=[checkpoint_en])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/500
91/91 [==============================] - 0s 3ms/sample - loss: 3.1259 - acc: 0.0440
Epoch 2/500
91/91 [==============================] - 0s 296us/sample - loss: 3.0823 - acc: 0.0659
Epoch 3/500
91/91 [==============================] - 0s 274us/sample - loss: 3.0355 - acc: 0.0769
Epoch 4/500
91/91 [==============================] - 0s 318us/sample - loss: 2.8909 - acc: 0.1868
Epoch 5/500
91/91 [==============================] - 0s 285us/sample - loss: 2.8354 - acc: 0.1319
Epoch 6/500
91/91 [==============================] - 0s 285us/sample - loss: 2.8012 - acc: 0.2198
Epoch 7/500
91/91 [==============================] - 0s 296us/sample - loss: 2.8004 - acc: 0.1209
Epoch 8/500
91/91 [==============================] - 0s 307us/sample - loss: 2.62

91/91 [==============================] - 0s 274us/sample - loss: 0.1914 - acc: 0.9560
Epoch 75/500
91/91 [==============================] - 0s 252us/sample - loss: 0.1095 - acc: 1.0000
Epoch 76/500
91/91 [==============================] - 0s 263us/sample - loss: 0.1933 - acc: 0.9451
Epoch 77/500
91/91 [==============================] - 0s 263us/sample - loss: 0.1287 - acc: 0.9670
Epoch 78/500
91/91 [==============================] - 0s 241us/sample - loss: 0.1155 - acc: 0.9780
Epoch 79/500
91/91 [==============================] - 0s 263us/sample - loss: 0.1862 - acc: 0.9451
Epoch 80/500
91/91 [==============================] - 0s 252us/sample - loss: 0.1478 - acc: 0.9560
Epoch 81/500
91/91 [==============================] - 0s 252us/sample - loss: 0.1817 - acc: 0.9341
Epoch 82/500
91/91 [==============================] - 0s 263us/sample - loss: 0.1403 - acc: 0.9780
Epoch 83/500
91/91 [==============================] - 0s 263us/sample - loss: 0.1813 - acc: 0.9560
Epoch 84/500
91/91 [===

91/91 [==============================] - 0s 263us/sample - loss: 0.0190 - acc: 1.0000
Epoch 157/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0543 - acc: 1.0000
Epoch 158/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0774 - acc: 0.9890
Epoch 159/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0786 - acc: 0.9780
Epoch 160/500
91/91 [==============================] - 0s 252us/sample - loss: 0.0518 - acc: 0.9890
Epoch 161/500
91/91 [==============================] - 0s 252us/sample - loss: 0.0363 - acc: 1.0000
Epoch 162/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0604 - acc: 0.9780
Epoch 163/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0271 - acc: 1.0000
Epoch 164/500
91/91 [==============================] - 0s 252us/sample - loss: 0.0873 - acc: 0.9780
Epoch 165/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0519 - acc: 1.0000
Epoch 166/500


91/91 [==============================] - 0s 263us/sample - loss: 0.0348 - acc: 0.9890
Epoch 239/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0989 - acc: 0.9670
Epoch 240/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0289 - acc: 0.9890
Epoch 241/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0273 - acc: 1.0000
Epoch 242/500
91/91 [==============================] - 0s 252us/sample - loss: 0.0471 - acc: 0.9890
Epoch 243/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0887 - acc: 0.9670
Epoch 244/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0463 - acc: 0.9890
Epoch 245/500
91/91 [==============================] - 0s 252us/sample - loss: 0.0422 - acc: 0.9890
Epoch 246/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0109 - acc: 1.0000
Epoch 247/500
91/91 [==============================] - 0s 252us/sample - loss: 0.0498 - acc: 0.9890
Epoch 248/500


91/91 [==============================] - 0s 252us/sample - loss: 0.0413 - acc: 0.9890
Epoch 321/500
91/91 [==============================] - 0s 274us/sample - loss: 0.0731 - acc: 0.9670
Epoch 322/500
91/91 [==============================] - 0s 252us/sample - loss: 0.0756 - acc: 0.9890
Epoch 323/500
91/91 [==============================] - 0s 274us/sample - loss: 0.0658 - acc: 0.9670
Epoch 324/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0638 - acc: 0.9890
Epoch 325/500
91/91 [==============================] - 0s 252us/sample - loss: 0.0506 - acc: 0.9780
Epoch 326/500
91/91 [==============================] - 0s 252us/sample - loss: 0.0673 - acc: 0.9890
Epoch 327/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0499 - acc: 0.9780
Epoch 328/500
91/91 [==============================] - 0s 252us/sample - loss: 0.0669 - acc: 0.9780
Epoch 329/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0398 - acc: 0.9890
Epoch 330/500


91/91 [==============================] - 0s 274us/sample - loss: 0.0772 - acc: 0.9890
Epoch 403/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0103 - acc: 1.0000
Epoch 404/500
91/91 [==============================] - 0s 252us/sample - loss: 0.0344 - acc: 0.9780
Epoch 405/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0517 - acc: 0.9670
Epoch 406/500
91/91 [==============================] - 0s 274us/sample - loss: 0.0486 - acc: 0.9890
Epoch 407/500
91/91 [==============================] - 0s 274us/sample - loss: 0.0717 - acc: 0.9780
Epoch 408/500
91/91 [==============================] - 0s 296us/sample - loss: 0.0216 - acc: 1.0000
Epoch 409/500
91/91 [==============================] - 0s 274us/sample - loss: 0.0745 - acc: 0.9560
Epoch 410/500
91/91 [==============================] - 0s 252us/sample - loss: 0.0144 - acc: 1.0000
Epoch 411/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0135 - acc: 1.0000
Epoch 412/500


91/91 [==============================] - 0s 274us/sample - loss: 0.0175 - acc: 1.0000
Epoch 485/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0081 - acc: 1.0000
Epoch 486/500
91/91 [==============================] - 0s 307us/sample - loss: 0.0378 - acc: 0.9890
Epoch 487/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0101 - acc: 1.0000
Epoch 488/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0098 - acc: 1.0000
Epoch 489/500
91/91 [==============================] - 0s 252us/sample - loss: 0.0175 - acc: 1.0000
Epoch 490/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0400 - acc: 0.9780
Epoch 491/500
91/91 [==============================] - 0s 241us/sample - loss: 0.0242 - acc: 0.9890
Epoch 492/500
91/91 [==============================] - 0s 263us/sample - loss: 0.0103 - acc: 1.0000
Epoch 493/500
91/91 [==============================] - 0s 252us/sample - loss: 0.0730 - acc: 0.9560
Epoch 494/500


### Train Spanish Model 

In [14]:
model_es = model_es()

optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_es.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
checkpoint_es = ModelCheckpoint(filepath='model_es.h5', monitor='acc')
model_es.fit(np.array(train_x_es), np.array(train_y_es), epochs=500, batch_size=5, verbose=1, callbacks=[checkpoint_es])

Epoch 1/500
52/52 [==============================] - 0s 4ms/sample - loss: 3.1389 - acc: 0.0577
Epoch 2/500
52/52 [==============================] - 0s 384us/sample - loss: 3.1358 - acc: 0.0192
Epoch 3/500
52/52 [==============================] - 0s 364us/sample - loss: 3.0127 - acc: 0.0769
Epoch 4/500
52/52 [==============================] - 0s 326us/sample - loss: 2.9720 - acc: 0.0769
Epoch 5/500
52/52 [==============================] - 0s 403us/sample - loss: 2.9125 - acc: 0.1538
Epoch 6/500
52/52 [==============================] - 0s 364us/sample - loss: 2.8637 - acc: 0.1923
Epoch 7/500
52/52 [==============================] - 0s 403us/sample - loss: 2.7689 - acc: 0.1923
Epoch 8/500
52/52 [==============================] - 0s 441us/sample - loss: 2.7519 - acc: 0.1731
Epoch 9/500
52/52 [==============================] - 0s 384us/sample - loss: 2.6503 - acc: 0.2308
Epoch 10/500
52/52 [==============================] - 0s 364us/sample - loss: 2.5669 - acc: 0.2308
Epoch 11/500
52/52 [=

52/52 [==============================] - 0s 441us/sample - loss: 0.5508 - acc: 0.8462
Epoch 84/500
52/52 [==============================] - 0s 364us/sample - loss: 0.1246 - acc: 1.0000
Epoch 85/500
52/52 [==============================] - 0s 441us/sample - loss: 0.2304 - acc: 0.9615
Epoch 86/500
52/52 [==============================] - 0s 364us/sample - loss: 0.2021 - acc: 0.9423
Epoch 87/500
52/52 [==============================] - 0s 345us/sample - loss: 0.2267 - acc: 0.9615
Epoch 88/500
52/52 [==============================] - 0s 384us/sample - loss: 0.1980 - acc: 0.9615
Epoch 89/500
52/52 [==============================] - 0s 422us/sample - loss: 0.2477 - acc: 0.9038
Epoch 90/500
52/52 [==============================] - 0s 384us/sample - loss: 0.2066 - acc: 0.9808
Epoch 91/500
52/52 [==============================] - 0s 441us/sample - loss: 0.1302 - acc: 1.0000
Epoch 92/500
52/52 [==============================] - 0s 364us/sample - loss: 0.1887 - acc: 0.9615
Epoch 93/500
52/52 [===

52/52 [==============================] - 0s 364us/sample - loss: 0.1054 - acc: 0.9615
Epoch 166/500
52/52 [==============================] - 0s 345us/sample - loss: 0.0963 - acc: 1.0000
Epoch 167/500
52/52 [==============================] - 0s 345us/sample - loss: 0.0864 - acc: 0.9808
Epoch 168/500
52/52 [==============================] - 0s 345us/sample - loss: 0.1230 - acc: 0.9615
Epoch 169/500
52/52 [==============================] - 0s 345us/sample - loss: 0.0838 - acc: 0.9808
Epoch 170/500
52/52 [==============================] - 0s 364us/sample - loss: 0.1352 - acc: 0.9423
Epoch 171/500
52/52 [==============================] - 0s 345us/sample - loss: 0.1082 - acc: 0.9808
Epoch 172/500
52/52 [==============================] - 0s 364us/sample - loss: 0.1310 - acc: 0.9808
Epoch 173/500
52/52 [==============================] - 0s 345us/sample - loss: 0.0661 - acc: 1.0000
Epoch 174/500
52/52 [==============================] - 0s 345us/sample - loss: 0.0532 - acc: 1.0000
Epoch 175/500


52/52 [==============================] - 0s 384us/sample - loss: 0.0406 - acc: 0.9808
Epoch 248/500
52/52 [==============================] - 0s 422us/sample - loss: 0.1135 - acc: 0.9808
Epoch 249/500
52/52 [==============================] - 0s 441us/sample - loss: 0.0254 - acc: 1.0000
Epoch 250/500
52/52 [==============================] - 0s 345us/sample - loss: 0.1358 - acc: 0.9423
Epoch 251/500
52/52 [==============================] - 0s 403us/sample - loss: 0.0867 - acc: 0.9615
Epoch 252/500
52/52 [==============================] - 0s 403us/sample - loss: 0.0721 - acc: 1.0000
Epoch 253/500
52/52 [==============================] - 0s 345us/sample - loss: 0.0676 - acc: 0.9808
Epoch 254/500
52/52 [==============================] - 0s 403us/sample - loss: 0.0475 - acc: 1.0000
Epoch 255/500
52/52 [==============================] - 0s 422us/sample - loss: 0.0874 - acc: 1.0000
Epoch 256/500
52/52 [==============================] - 0s 403us/sample - loss: 0.0626 - acc: 0.9808
Epoch 257/500


52/52 [==============================] - 0s 479us/sample - loss: 0.0419 - acc: 0.9808
Epoch 330/500
52/52 [==============================] - ETA: 0s - loss: 0.0107 - acc: 1.000 - 0s 422us/sample - loss: 0.0190 - acc: 1.0000
Epoch 331/500
52/52 [==============================] - 0s 499us/sample - loss: 0.0295 - acc: 1.0000
Epoch 332/500
52/52 [==============================] - 0s 422us/sample - loss: 0.0229 - acc: 1.0000
Epoch 333/500
52/52 [==============================] - 0s 364us/sample - loss: 0.0466 - acc: 0.9808
Epoch 334/500
52/52 [==============================] - 0s 403us/sample - loss: 0.0834 - acc: 0.9808
Epoch 335/500
52/52 [==============================] - 0s 384us/sample - loss: 0.0349 - acc: 1.0000
Epoch 336/500
52/52 [==============================] - 0s 441us/sample - loss: 0.0457 - acc: 1.0000
Epoch 337/500
52/52 [==============================] - 0s 403us/sample - loss: 0.0225 - acc: 1.0000
Epoch 338/500
52/52 [==============================] - 0s 364us/sample - los

52/52 [==============================] - 0s 422us/sample - loss: 0.0414 - acc: 0.9808
Epoch 411/500
52/52 [==============================] - 0s 422us/sample - loss: 0.1345 - acc: 0.9423
Epoch 412/500
52/52 [==============================] - 0s 422us/sample - loss: 0.0621 - acc: 0.9808
Epoch 413/500
52/52 [==============================] - 0s 383us/sample - loss: 0.0366 - acc: 0.9808
Epoch 414/500
52/52 [==============================] - 0s 384us/sample - loss: 0.0519 - acc: 0.9808
Epoch 415/500
52/52 [==============================] - 0s 364us/sample - loss: 0.0613 - acc: 0.9808
Epoch 416/500
52/52 [==============================] - 0s 364us/sample - loss: 0.0302 - acc: 0.9808
Epoch 417/500
52/52 [==============================] - 0s 364us/sample - loss: 0.0374 - acc: 1.0000
Epoch 418/500
52/52 [==============================] - 0s 364us/sample - loss: 0.0153 - acc: 1.0000
Epoch 419/500
52/52 [==============================] - 0s 345us/sample - loss: 0.0822 - acc: 0.9615
Epoch 420/500


52/52 [==============================] - 0s 384us/sample - loss: 0.0325 - acc: 1.0000
Epoch 492/500
52/52 [==============================] - 0s 384us/sample - loss: 0.0353 - acc: 0.9808
Epoch 493/500
52/52 [==============================] - 0s 384us/sample - loss: 0.0628 - acc: 0.9615
Epoch 494/500
52/52 [==============================] - 0s 422us/sample - loss: 0.0224 - acc: 1.0000
Epoch 495/500
52/52 [==============================] - 0s 422us/sample - loss: 0.0699 - acc: 0.9423
Epoch 496/500
52/52 [==============================] - 0s 384us/sample - loss: 0.0337 - acc: 0.9808
Epoch 497/500
52/52 [==============================] - 0s 345us/sample - loss: 0.0149 - acc: 1.0000
Epoch 498/500
52/52 [==============================] - 0s 364us/sample - loss: 0.0272 - acc: 1.0000
Epoch 499/500
52/52 [==============================] - 0s 384us/sample - loss: 0.0220 - acc: 1.0000
Epoch 500/500
52/52 [==============================] - 0s 460us/sample - loss: 0.0066 - acc: 1.0000


## Preprocess User Query

In [38]:
# performs preprocessing on inputted text (lower casing and lemmatization)
def preprocess_input(text):
    text_token = nltk.word_tokenize(text)
    text_token = [lem.lemmatize(word.lower()) for word in text_token]
    return text_token

# coverts preprocessed text to a matrix form 
def bow_en(text_token):
    bag = []
    for word in words_en:
        if word in text_token:
            bag.append(1)
        else:
            bag.append(0)
    return bag

def bow_es(text_token):
    bag = []
    for word in words_es:
        if word in text_token:
            bag.append(1)
        else:
            bag.append(0)
    return bag

# inputs user query in matrix form and returns a predicted label 
def predict_label_en(bow_en):
    prediction = model_en.predict(np.array(bow_en).reshape((1, len(words_en))))
    max_value = np.max(prediction)
    if max_value > 0.7:
        predict_label = labels_en[np.argmax(prediction)]
    else:
        predict_label = 'NA'
    return predict_label

def predict_label_es(bow_es):
    prediction = model_es.predict(np.array(bow_es).reshape((1, len(words_es))))
    max_value = np.max(prediction)
    if max_value > 0.7:
        predict_label = labels_es[np.argmax(prediction)]
    else:
        predict_label = 'NA'
    return predict_label


## Response Generation

### English Responses

In [39]:
responses_en = { 
        'cancel' : "Ok, it's cancelled. Once you're ready, please type *`daily`* to fill out your follow-up reports.",

        'chicken' : "Here comes Arthur!\n\n*[Launch Rubber Chicken]({{{url}}})* 🐔\n\n_Pro tip: you can change the last part of Arthur's URL in your browser in order to create a unique room.",

        'daily' :  "No response",
            
        'dailystatus' : "No response",
            
        'dashboard' :"Ok, please *[follow this link]({{{url}}})* to access the web dashboard.",

        'datetime' :  "The time information for your user is:",
            
        'disable' : "Ok, it's done. You have disconnected and disabled all private messaging between you and me.\nYou can still use the [web application]({{{url}}}).\n\nIf you want to talk to me again, just type *`enable`*",


        'enable' : "It seems that you had enabled me previously, I'm ready to help you.\nPlease type *`help`* to know more about how I can help you.",
                    # 8. "Hey! Welcome back, I'm ready to help you.\nYou can *[login into the web application]({{{url}}})* to check your settings."
            
        'extras' :"Hey, your wish is my command, and these are the extra commands",
            
        'feedback' : "Hey *[user]*, it'd be great to get some feedback from you. If it's something important, I recommend you to visit our support center.",

        'hello' : "Hey [user]! Have a great day and remember I'm so happy about being your friend.\nType *`help`* to learn more about me.",

        'help' : "Hey, your wish is my command, and these are my available commands:",

        'kudos' :  "kudos response: ",
        
        'news' :  "Take a look at *[my blog](https://www.dailybot.co/blog/)* to read about tips, key learnings and other content that our team is sharing.",

        'report' : 'No response',
        
        'resetdaily' :"This command has been changed, please type *`help`* to see all current commands.",

        'settings' : "This is a quick overview of your settings. You can manage these settings on my *[Web Application]({{{url}}})*:",

        'skipdaily' :"no response",
        
        'snooze' : "Ok, I’ll remind you in *{{time}} minutes*",

        'textformat' : "Hi there! The following is a list of the different formats I support.\nYou can also see this *[support article](https://www.dailybot.com)* to know more about how to build responses with these formats.",

        'thanks' : ["Oh, don't you even mention it. I'm always happy to help.",
                                "It's always my pleasure.",
                                "You are so very welcome.",
                                "It's no trouble at all. 😊",
                                "Anytime! it's always my pleasure. 😊"],
       
        'timezone' : "The time information for the typed timezone is:",

        'NA':  "Sorry for not understanding. Please type *`help`* for me to help you."
        
        
        
    }

### Spanish Responses

In [40]:
responses_es = { 
        'cancel' : "Ok, has cancelado tu seguimiento diario. Por favor escribe *`daily`* para empezar cuando desees.",
    
        'chicken' : "¡Aquí viene Arthur!\n\n[Abrir Rubber Chicken]({{{url}}}) 🐔\n\n_Pro tip: puedes cambiar la última parte de la URL de Arthur en tu navegador y así tendrás una sala única._",

        'daily' :  "No hay respuesta",
        
        'dailystatus' :  "No hay respuesta",
        
        'dashboard' : "Ok, *[abre este enlace]({{{url}}})* para abrir la web y dashboard.",

        'datetime' : "La información horaria de tu usuario es:",

        'disable' : "Listo, está hecho. Has desconectado y deshabilitado todos los mensajes privados entre tu y yo.\nPor cierto, puedes seguir utilizando mi [aplicación web]({{{url}}}).\n\nSi deseas volver a hablar conmigo simplemente escribe *`enable`*.",

        'enable' : "Parece que ya me habías habilitado previamente, estoy listo para ayudarte.\nPor favor, escribe *`help`* para saber más sobre cómo puedo ayudarte.",
                    # 8. "¡Hola! Bienvenido de nuevo, estoy listo para ayudarte.\nPor favor inicia sesión en mi *[aplicación web]({{{url}}})* para confirmar su configuración."
        
        'extras' : "Hola, puedes usar estos comandos extras para interactuar conmigo:",

        
        'feedback' : "Hola *[user]*!, Sería grandióso recibir tu feedback. Si es algo importante, Te recomiendo visitar nuestro centro de soporte.",

        
        'hello' :  "¡Hola [user]! Ten un excelente día y recuerda que estoy muy feliz de ser tu amigo.\nEscribe *`help`* para aprender más sobre mí.",

        
        'help' : "Hola, puedes usar estos comandos para interactuar conmigo:",
        
        'kudos' :"kudos response: ",
        
        'news' : "Echa un vistazo a *[mi blog](https://www.dailybot.com/blog/)* (en inglés) para leer sobre trucos, aprendizajes y otro contenido que comparte nuestro equipo.",

        
        'report' : "No hay respuesta",
        
        'resetdaily' : "Este comando ha cambiado, por favor escribe *`help`* para ver todos los comandos actuales.",

        'settings' : "Este es un resúmen de tu configuración. Puedes gestionar la configuración en mi *[aplicación web]({{{url}}})*:",

        'skipdaily' : "No hay respuesta",
        
        'snooze' : "Ok, te recordaré en *{{time}} minutos*.",

        'textformat' : "Hola! La siguiente es una lista de los diferentes formatos que soporto.\nPuedes verlo aquí *[artículo de apoyo](https://www.dailybot.com)* para conocer más acerca de como construir respuestas con estos formatos.",

        'thanks' : ["¡Con muuucho gusto!",
                                 "Siempre es un placer.",
                                 "Estoy para ayudarte.",
                                 "A ti por tu tiempo. 😊",
                                 "¡Cuando me necesites! siempre es un placer. 😊"],
    
        'timezone' :  "La información de la zona horaria introducida es:",

        'NA': "Lo siento por no entender. Por favor escribe *`help`* para ayudarte."        
        
    }

In [41]:
# inputs the user intent and returns the appropriate response
def generate_response_en(query_intent):

        return responses_en[query_intent]
    
# inputs the user intent and returns the appropriate response
def generate_response_es(query_intent):

        return responses_es[query_intent]

## Saving unidentified Queries

In [ ]:
# Create list of queries with no response
try:
    no_intent_queries = open("no_intent_queries.txt", "x")
except:
    pass

# Chatbot

In [48]:
prediction = predict_label_en(np.array(bow_en(preprocess_input('feedback please'))))
prediction

print('Insert command here (type "quit" to stop chatbot)\n')

while 1 >0:
    
    input_text = input()
    if input_text=='quit':
        break
    if len(input_text) < 3:
        while len(input_text) <3:
            input_text = input_text + " "
    prediction = predict_label(np.array(bow(preprocess_input(input_text))))
    response = generate_response(prediction)
    if prediction == 'NA':
            queries_list = queries_list + input_text+'\n'    
    

    if TextBlob(input_text).detect_language() == 'en':
        if prediction == 'thanks':
            print(random.choice(response['en']))
        else:
            print(response['en'])
    elif TextBlob(input_text).detect_language() == 'es' :
        if prediction == 'thanks':
            print(random.choice(response['es'])+'\n')
        else:
            print(response['es']+'\n')
    else:
        print("Sorry for not understanding. Please type *`help`* for me to help you.")
    print(' ')
        

'feedback'

## Add "NA" Queries to .txt File

In [52]:
spell = Speller()
spell('Give me feedbacs')

'Give me feedback'